### Time Estimate


In [1]:
args = {
    "dataset_folder": "./train_10M",
    "gradients_per_file" : 10000,
    "epochs" : 10,
}

In [18]:
total_in_h = 0


In [42]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("./TestModel", max_len=512)

/data/loriss21dm/TracInVenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [47]:
from datasets import load_dataset
import datasets
t = lambda x : tokenizer(x["text"], return_special_tokens_mask=True, truncation=True, max_length=512)

dataset = datasets.load_from_disk("curricula/datasets/test_eval")
dataset.set_transform(t)





In [48]:
from transformers import DataCollatorForLanguageModeling

In [79]:
import util
data_collator = util.DeterministicDataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
data_collator.set_epoch(1000)
# https://github.com/ayoolaolafenwa/TrainNLP
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}


eval_dataset = dataset.map(insert_random_mask,batched=True,
)

eval_dataset = eval_dataset.rename_columns({"masked_input_ids": "input_ids",
"masked_attention_mask": "attention_mask","masked_labels": "labels"})

In [80]:
eval_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 117
})

In [74]:
 dataset.map

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 117
})

#### if.py

In [19]:
args["num_processes"] = 4
TIME_PER_BATCH_IN_MIN = 4

In [20]:
time_if = ((((len(dataset["train"]) / args["gradients_per_file"]) *args["epochs"]) /args["num_processes"]) * TIME_PER_BATCH_IN_MIN)/60
print(time_if, "hours")
total_in_h+=time_if

19.650233333333333 hours


In [21]:
SIZE_IN_GB_OF_CHUNK = 7.4


In [22]:
print((len(dataset["train"]) / args["gradients_per_file"]) *args["epochs"] * SIZE_IN_GB_OF_CHUNK/1000, "TB disk space")

8.7247036 TB disk space


#### merge.py

In [7]:
args["batch_size"] = 20
args["num_processes"] = 4

SIZE_IN_GB_OF_CHUNK = 7.4 # ls
time_single_chunk = 15.36 # get from log
time_load_task = 110 # get from log

In [8]:
print("RAM requirements", (args["batch_size"]+1)*args["num_processes"]*SIZE_IN_GB_OF_CHUNK, "GB")

RAM requirements 621.6 GB


In [9]:


num_chunks =(((len(dataset["train"]) / args["gradients_per_file"])))
num_batches = num_chunks / args["batch_size"]

time_single_batch = (num_chunks * time_single_chunk) + time_load_task
time_single_epoch = (num_batches * time_single_batch) 
print("Time for a single epoch and one process", time_single_epoch /60/60, "hours")
total_merge = (time_single_epoch * args["epochs"] /args["num_processes"])/60/60
print("Total time with multiprocessing", total_merge, "hours")
total_in_h += total_merge

Time for a single epoch and one process 3.1456183649070217 hours
Total time with multiprocessing 3.9320229561337774 hours


## Total

In [10]:
print(total_in_h, "hours")

13.757139622800445 hours
